In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

import tensorflow as tf
# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

import pyaudio
import wave
import glob
%matplotlib inline

In [12]:
n_dim = 13 #X_train.shape[1]
n_classes = 2
n_hidden_units_one = 280 
n_hidden_units_two = 300
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.01

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 1.0
WAVE_OUTPUT_FILENAME = "../data/output.wav"

In [ ]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,n_classes])

W_1 = tf.Variable(tf.random_normal([n_dim,n_hidden_units_one], mean = 0, stddev=sd))
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean = 0, stddev=sd))
h_1 = tf.nn.tanh(tf.matmul(X,W_1) + b_1)


W_2 = tf.Variable(tf.random_normal([n_hidden_units_one,n_hidden_units_two], mean = 0, stddev=sd))
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean = 0, stddev=sd))
h_2 = tf.nn.sigmoid(tf.matmul(h_1,W_2) + b_2)


W = tf.Variable(tf.random_normal([n_hidden_units_two,n_classes], mean = 0, stddev=sd))
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd))
y_ = tf.nn.softmax(tf.matmul(h_2,W) + b)

init = tf.global_variables_initializer()

In [14]:
cost_function = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y_), reduction_indices=[1])) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
def mfcc(raw, chunk_size=8192, sr=44100, n_mfcc=13):
    mfcc = np.empty((13, 0))
    for i in range(0, len(raw), chunk_size):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc)
        mfcc = np.hstack((mfcc, mfcc_slice))
    return mfcc

In [29]:
model_path_f = '../models/FCNN/'
filename = 'my_test_model.meta'

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loader = tf.train.import_meta_graph(model_path_f+filename)
loader.restore(sess, tf.train.latest_checkpoint(model_path_f))

SR = 44100
sound_path = WAVE_OUTPUT_FILENAME
####
while(1):
    '''
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,channels = CHANNELS,rate = RATE,input = True,frames_per_buffer = CHUNK)

    #print("start to record the audio.")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    #print("Recording finished.")
    stream.stop_stream()
    stream.close()

    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    ####
    '''
    sample, sample_rate = librosa.load("../data/phantom/JUNE_01_PHANTOMS/wavs/purdue_P2_unloaded_up_down.wav",SR)
    #print(sample.shape)
    #rint(sample.shape)
    mfcc_sample = mfcc(sample)

    '''
    # Let's pad on the first and second deltas while we're at it
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    print(delta2_mfcc.shape , sample_rate)
    '''
    #dataX = mfcc.T
    #print('delta shape:',dataX.shape)
    
    #X_input = np.array(dataX[dataX.shape[0]-1:])
    X_input = mfcc_sample.T
    #X_input = X_input.reshape(1,13)
    #print('X_input shape:',X_input.shape)
    #print('\n\n\n')
    print(X_input.shape)
    y_pred = None
    y_pred = sess.run(tf.argmax(y_,1),feed_dict={X: X_input})
    #rint(y_pred)
    print(np.sum(y_pred==1))

sess.close()

c:\users\jh\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning:

An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).



INFO:tensorflow:Restoring parameters from ../models/FCNN/my_test_model
(1836, 13)
1830
(1836, 13)
1830
(1836, 13)
1830
(1836, 13)
1830
(1836, 13)
1830
(1836, 13)
1830


KeyboardInterrupt: 